#Install dependencias

In [1]:
!pip install google-cloud
!pip install mysql-connector-python
!pip install -U -q PyDrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Funcion: **list_mysql_tables**

In [2]:
import mysql
import pandas as pd
import mysql.connector as connection
 
import warnings
warnings.filterwarnings('ignore')

def list_mysql_tables(request):
  
  headers = {
      'Access-Control-Allow-Origin': '*',
      'Content-Type': 'application/json'
  }

  db = mysql.connector.connect(
    host="72.167.124.128",
    user="user_test2023",
    password="g&d*nf!12vQj",
    database="db_test2023"
  )
   
  query = '''SELECT
	            table_schema
              , table_name
              , table_rows
            FROM information_schema.tables
            WHERE TABLE_TYPE = "BASE TABLE"'''
 
  result_dataframe = pd.read_sql(query, db)
  db.close()
  
  return (result_dataframe.to_json(orient='records'), 200, headers)



**Test**

In [3]:
# Test
class RequestArgsMock:
    def __init__(self, **kwargs):
        self.args = kwargs

request = RequestArgsMock(cliente='128')
print(list_mysql_tables(request))

('[{"table_schema":"db_test2023","table_name":"customers","table_rows":122},{"table_schema":"db_test2023","table_name":"employees","table_rows":23},{"table_schema":"db_test2023","table_name":"offices","table_rows":7},{"table_schema":"db_test2023","table_name":"orderdetails","table_rows":2996},{"table_schema":"db_test2023","table_name":"orders","table_rows":326},{"table_schema":"db_test2023","table_name":"payments","table_rows":273},{"table_schema":"db_test2023","table_name":"productlines","table_rows":7},{"table_schema":"db_test2023","table_name":"products","table_rows":110}]', 200, {'Access-Control-Allow-Origin': '*', 'Content-Type': 'application/json'})


**Fix para credenciales**
Dependencias

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials# Authenticate and create the PyDrive client.

Credenciales

In [5]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':'1B_iIqKrhT43T5JLKSIqfxbEwx998K6T9'}) 
downloaded.GetContentFile('secrets.json')  


# Funcion: **copy_table_mysql_to_bigquery**

In [6]:
import os
import json
import pandas as pd
import mysql
import mysql.connector as connection
from google.cloud import bigquery

import warnings
warnings.filterwarnings('ignore')


def copy_table_mysql_to_bigquery(request):
  table_source = request.args['table_source']
  table_destination = request.args['table_destination']
  project_id = 'test-prj-data-911'
  schema = 'raw'
  table_id =  '{0}.{1}.{2}'.format(project_id, schema, table_destination)

  headers = {
      'Access-Control-Allow-Origin': '*',
      'Content-Type': 'application/json'
  }

  if table_source.strip() == "" or table_destination.strip() == "":
    return (json.dumps({'result': 'error', 'description': 'no table name'}), 200, headers)
  
  db = mysql.connector.connect(
    host="72.167.124.128",
    user="user_test2023",
    password="g&d*nf!12vQj",
    database="db_test2023"
  )
   
  query = 'SELECT * FROM {0}'.format(table_source)
  result_dataframe = pd.read_sql(query, db)
  db.close()
  
  os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "secrets.json"

  client = bigquery.Client()

  job_config = bigquery.LoadJobConfig(
      write_disposition="WRITE_TRUNCATE",
  )


  job = client.load_table_from_dataframe(
    result_dataframe, table_id, job_config=job_config
  )
  job.result()

  try:
      client.get_table(table_id)
  except NotFound:
      return (json.dumps({'result': 'error', 'description': 'table not found'}), 200, headers)

  return (json.dumps({'result': 'ok'}), 200, headers)

Test

In [7]:
# Test
class RequestArgsMock:
    def __init__(self, **kwargs):
        self.args = kwargs

request = RequestArgsMock(table_source='employees', table_destination='employees')
print(copy_table_mysql_to_bigquery(request))

('{"result": "ok"}', 200, {'Access-Control-Allow-Origin': '*', 'Content-Type': 'application/json'})


# Funcion: **get_rows_from_bigquery**

In [8]:
import pandas as pd
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import os
import json


def get_rows_from_bigquery(request):
 
  table_name = request.args['table_name']
  project_id = 'test-prj-data-911'
  schema = 'raw'
  table_id =  '{0}.{1}.{2}'.format(project_id, schema, table_name)
  headers = {
      'Access-Control-Allow-Origin': '*',
      'Content-Type': 'application/json'
  }

  if table_name.strip() == "":
    return (json.dumps({'result': 'error', 'description': 'no table name','status': 200}), 200, headers)
  
  os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "secrets.json"

  client = bigquery.Client()
  try:
      client.get_table(table_id)
  except NotFound:
      return (json.dumps({'result': 'error', 'description': 'table not found','status': 200}), 200, headers)
      
  query = 'SELECT COUNT(*) filas FROM `{0}`.`{1}`'.format(schema, table_name)
  query_job = client.query(query)
  query_job.result()

  rows = list(query_job)
  return (json.dumps({'result': 'ok', 'rows': rows[0].filas, 'status': 200}), 200, headers)


**Test**

In [9]:
# Test
class RequestArgsMock:
    def __init__(self, **kwargs):
        self.args = kwargs

request = RequestArgsMock(table_name='employees')
print(get_rows_from_bigquery(request))

('{"result": "ok", "rows": 23, "status": 200}', 200, {'Access-Control-Allow-Origin': '*', 'Content-Type': 'application/json'})
